## Parte 1: Carregamento e Limpeza de Dados

Para começar, carregamos nossa principal ferramenta de trabalho, a biblioteca pandas, e em seguida, importamos as 4 planilhas de dados (partidas, estatísticas, cartões e gols).

In [1]:
import pandas as pd

In [2]:
try:
    df_full = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-full.csv')
    df_stats = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-estatisticas-full.csv')
    df_cartoes = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-cartoes.csv')
    df_gols = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-gols.csv')

    print("Os 4 arquivos CSV foram carregados com sucesso!")
    
except FileNotFoundError as e:
    print(f"Erro: O arquivo {e.filename} não foi encontrado.")
    print("Por favor, certifique-se de que os arquivos CSV estão no mesmo diretório que o seu notebook.")

Os 4 arquivos CSV foram carregados com sucesso!


In [3]:
# Filtro de Qualidade em df_stats

print(f"Número de linhas em df_stats ANTES do filtro: {len(df_stats)}")

# Lista de colunas de estatísticas numéricas que indicam atividade em um jogo
colunas_de_atividade = ['chutes', 'faltas', 'cartao_amarelo', 'cartao_vermelho', 'impedimentos', 'escanteios']

# Calculamos a soma das estatísticas para cada linha. Se a soma for 0, significa que não há dados.
soma_stats = df_stats[colunas_de_atividade].sum(axis=1)

# Mantemos apenas as linhas onde a soma é maior que 0
df_stats_com_dados = df_stats[soma_stats > 0].copy()

print(f"Número de linhas em df_stats DEPOIS do filtro: {len(df_stats_com_dados)}")
print("\nFiltro de qualidade aplicado com sucesso! Apenas partidas com dados de estatísticas foram mantidas.")

Número de linhas em df_stats ANTES do filtro: 17570
Número de linhas em df_stats DEPOIS do filtro: 6820

Filtro de qualidade aplicado com sucesso! Apenas partidas com dados de estatísticas foram mantidas.


Ajuste de Data e Hora, além de renomear a coluna ID em df_full para 'partida_id', padronizando

In [4]:
# Ajustando os tipos de dados de data e hora
df_full['data'] = pd.to_datetime(df_full['data'], format='%d/%m/%Y', errors='coerce')
df_full['hora'] = pd.to_datetime(df_full['hora'], format='%H:%M', errors='coerce').dt.time

# Renomeando a coluna 'ID' para 'partida_id' para padronização
df_full.rename(columns={'ID': 'partida_id'}, inplace=True)

print("Formatos de data/hora e coluna de ID ajustados com sucesso.")

Formatos de data/hora e coluna de ID ajustados com sucesso.


#### Sincronizando as Tabelas, garantindo qualidade

Para garantir a máxima confiabilidade, vamos trabalhar apenas com partidas que possuem registros completos em todas as quatro tabelas. Isso evita análises baseadas em dados incompletos.

In [5]:
# Encontrando os IDs de partida comuns a todas as tabelas
ids_full = set(df_full['partida_id'])
ids_stats = set(df_stats_com_dados['partida_id'])
ids_cartoes = set(df_cartoes['partida_id'])
ids_gols = set(df_gols['partida_id'])

# Encontra a interseção de IDs
common_ids = set.intersection(ids_full, ids_stats, ids_cartoes, ids_gols)

print(f"Número de partidas em comum a todas as 4 bases: {len(common_ids)}")

Número de partidas em comum a todas as 4 bases: 3059


Agora, filtramos nossas tabelas para manter apenas essas partidas "validadas".

In [6]:
# Filtrando os DataFrames para manter apenas os dados completos
common_ids_list = list(common_ids)

df_full_cleaned = df_full[df_full['partida_id'].isin(common_ids_list)].copy()
df_stats_cleaned = df_stats_com_dados[df_stats_com_dados['partida_id'].isin(common_ids_list)].copy()
df_cartoes_cleaned = df_cartoes[df_cartoes['partida_id'].isin(common_ids_list)].copy()
df_gols_cleaned = df_gols[df_gols['partida_id'].isin(common_ids_list)].copy()

print("Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):")
print(f"Partidas:      {len(df_full)} -> {len(df_full_cleaned)}")
print(f"Estatísticas:  {len(df_stats_com_dados)} -> {len(df_stats_cleaned)}")
print(f"Cartões:       {len(df_cartoes)} -> {len(df_cartoes_cleaned)}")
print(f"Gols:          {len(df_gols)} -> {len(df_gols_cleaned)}")

Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):
Partidas:      8785 -> 3059
Estatísticas:  6820 -> 6118
Cartões:       20953 -> 15659
Gols:          9861 -> 7928


#### Tratando Campos Vazios

O último passo da limpeza é preencher campos vazios (NaN) de forma inteligente para evitar erros e inconsistências. Aqui, não apagamos os dados NaN pois são grande parte do nosso banco de dados e dados como 'posse de bola', serão usados em análises futuras. Sobre as formações dos times, como são poucos dados e não pretendo usar, 'dropei' essas colunas.

In [7]:
# Removendo as colunas de formação do DataFrame df_full_cleaned
df_full_cleaned.drop(columns=['formacao_mandante', 'formacao_visitante'], inplace=True, errors='ignore')
# Em df_stats, estatísticas ausentes viram 'Sem Info' (serão tratadas depois
df_stats_cleaned['posse_de_bola'].fillna('Sem Info', inplace=True)
df_stats_cleaned['precisao_passes'].fillna('Sem Info', inplace=True)

# Em df_cartoes, removemos a coluna 'num_camisa' e preenchemos 'posicao'
df_cartoes_cleaned.drop(columns=['num_camisa'], inplace=True, errors='ignore')
df_cartoes_cleaned['posicao'].fillna('Sem Info', inplace=True)

# Em df_gols, um gol sem tipo definido é assumido como 'Gol Normal'
df_gols_cleaned['tipo_de_gol'].fillna('Gol Normal', inplace=True)

print("Campos vazios tratados com sucesso.")

Campos vazios tratados com sucesso.


C:\Users\rossi\AppData\Local\Temp\ipykernel_18032\3709112189.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_stats_cleaned['posse_de_bola'].fillna('Sem Info', inplace=True)
C:\Users\rossi\AppData\Local\Temp\ipykernel_18032\3709112189.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

## Parte 2: Construindo a Tabela Mestra  (EM MANUTENÇÃO)
Com os dados limpos e sincronizados, o objetivo agora é criar uma única tabela onde cada linha representa uma partida e contém as estatísticas do time da casa e do visitante, lado a lado

#### Separando Estatísticas: Mandante vs. Visitante

Primeiro, separamos os dados da tabela de estatísticas em duas novas tabelas: uma só para os times da casa (mandantes) e outra só para os visitantes.

In [8]:
# Separando as estatísticas
stats_com_times = pd.merge(df_stats_cleaned, df_full_cleaned[['partida_id', 'mandante', 'visitante']], on='partida_id')
mandante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['mandante']].copy()
visitante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['visitante']].copy()

print("Estatísticas separadas para mandantes e visitantes.")

Estatísticas separadas para mandantes e visitantes.


In [9]:
mandante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
0,4741,13,Chapecoense,15,0,35%,280,Sem Info,19,4,0,0,8,Chapecoense,Flamengo
3,4808,20,Palmeiras,10,0,44%,384,Sem Info,18,4,0,1,5,Palmeiras,Criciuma
5,4833,22,Coritiba,9,0,43%,360,Sem Info,14,5,0,1,11,Coritiba,Sao Paulo


In [10]:
visitante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
1,4741,13,Flamengo,9,0,65%,509,Sem Info,13,1,0,1,5,Chapecoense,Flamengo
2,4808,20,Criciuma,10,0,56%,471,Sem Info,17,3,0,1,6,Palmeiras,Criciuma
4,4833,22,Sao Paulo,5,0,57%,481,Sem Info,10,1,0,3,2,Coritiba,Sao Paulo


Para que na tabela final saibamos exatamente a qual time cada estatística pertence, adicionamos os prefixos mandante_ e visitante_ às colunas.

In [11]:
# Adicionando prefixos
colunas_stats = [
    'chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 'precisao_passes',
    'faltas', 'cartao_amarelo', 'cartao_vermelho', 'impedimentos', 'escanteios'
]

# Prefixando e selecionando colunas para a tabela de mandantes
mandante_stats_renamed = mandante_stats[colunas_stats].add_prefix('mandante_')
mandante_stats_renamed['partida_id'] = mandante_stats['partida_id']

# Prefixando e selecionando colunas para a tabela de visitantes
visitante_stats_renamed = visitante_stats[colunas_stats].add_prefix('visitante_')
visitante_stats_renamed['partida_id'] = visitante_stats['partida_id']

print("Prefixos adicionados com sucesso!")
display(mandante_stats_renamed.head(300))

Prefixos adicionados com sucesso!


,mandante_chutes,mandante_chutes_no_alvo,mandante_posse_de_bola,mandante_passes,mandante_precisao_passes,mandante_faltas,mandante_cartao_amarelo,mandante_cartao_vermelho,mandante_impedimentos,mandante_escanteios,partida_id
0,15,0,35%,280,Sem Info,19,4,0,0,8,4741
3,10,0,44%,384,Sem Info,18,4,0,1,5,4808
5,9,0,43%,360,Sem Info,14,5,0,1,11,4833
7,15,0,42%,357,Sem Info,20,2,0,1,9,4846
8,15,0,52%,394,Sem Info,16,3,0,5,2,4856
...,...,...,...,...,...,...,...,...,...,...,...
590,17,0,44%,363,Sem Info,14,4,0,0,8,5305
592,17,0,59%,552,Sem Info,15,2,0,0,6,5306
595,16,0,57%,541,Sem Info,10,1,0,3,4,5307
596,10,0,51%,315,Sem Info,16,3,0,1,4,5308


In [12]:
df_stats_unificado = pd.merge(mandante_stats_renamed, visitante_stats_renamed, on='partida_id')

# Célula 11: Junção final com a tabela principal
df_final_partidas = pd.merge(df_full_cleaned, df_stats_unificado, on='partida_id')

print("Tabela Mestra final criada com sucesso!")
print(f"Dimensões da tabela final: {df_final_partidas.shape}")

Tabela Mestra final criada com sucesso!
Dimensões da tabela final: (3059, 34)


In [13]:
display(df_final_partidas.head())

,partida_id,rodata,data,hora,mandante,visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,...,visitante_chutes,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios
0,4741,13,2014-08-03,16:00:00,Chapecoense,Flamengo,C. Rodrigues,V. Luxemburgo da Silva,Chapecoense,Arena Condá,...,9,0,65%,509,Sem Info,13,1,0,1,5
1,4808,20,2014-09-10,19:30:00,Palmeiras,Criciuma,D. Silvestre Júnior,G. Dal Pozzo,Palmeiras,Estádio Municipal Paulo Machado de Carvalho,...,10,0,56%,471,Sem Info,17,3,0,1,6
2,4833,22,2014-09-17,22:00:00,Coritiba,Sao Paulo,M. dos Santos Gonçalves,M. Ramalho,Coritiba,Couto Pereira,...,5,0,57%,481,Sem Info,10,1,0,3,2
3,4846,23,2014-09-21,18:30:00,Gremio,Chapecoense,L. Scolari,J. da Silva,Gremio,Arena do Grêmio,...,14,0,58%,477,Sem Info,14,4,0,2,4
4,4856,24,2014-09-25,20:29:00,Atletico-MG,Santos,L. Culpi,E. Alves Moreira,Atletico-MG,Estádio Raimundo Sampaio,...,13,0,48%,355,Sem Info,18,3,0,6,10


### Salvando Tabela Mestra

In [14]:
# Salvando a tabela final em um novo arquivo
df_final_partidas.to_csv('Brasileirao_Dataset/partidas_com_estatisticas_completas.csv', index=False)

print("Arquivo 'partidas_com_estatisticas_completas.csv' salvo com sucesso!")

Arquivo 'partidas_com_estatisticas_completas.csv' salvo com sucesso!
